In [1]:
import asyncio

from src.sf_auto_eval import evaluate_patches
import nest_asyncio

if __name__ == "__main__":
    nest_asyncio.apply()
    # Example usage:
    bug_name_example = "Chart-10"
    candidate_patches = [
      "public String generateToolTipFragment(String toolTipText) {\n    return \" title=\\\"\" + toolTipText.replace(\"\\\"\", \"&quot;\") + \"\\\" alt=\\\"\\\"\";\n}",
      "import org.apache.commons.text.StringEscapeUtils;\n\npublic String generateToolTipFragment(String toolTipText) {\n    return \" title=\\\"\" + StringEscapeUtils.escapeHtml4(toolTipText) + \"\\\" alt=\\\"\\\"\";\n}",
      "public String generateToolTipFragment(String toolTipText) {\n    StringBuilder sb = new StringBuilder();\n    sb.append(\" title=\\\"\");\n    sb.append(toolTipText.replace(\"\\\"\", \"&quot;\"));\n    sb.append(\"\\\" alt=\\\"\\\"\");\n    return sb.toString();\n}"
    ]

    # Run the evaluation in an async event loop
    all_results = asyncio.run(evaluate_patches(bug_name_example, candidate_patches))

    # Inspect the results
    for idx, result in enumerate(all_results, start=1):
        print(f"\n----- Result for Patch {idx} -----")
        # print("Prompt:\n", result["prompt"])
        # print("Analysis:\n", result["analysis"])
        print("\n", result)



----- Result for Patch 1 -----

 Chart-10


In [2]:
print(all_results)

{'Chart-10': [{'prompt': 'You are an expert at evaluating program patches for correctness.\n\nYou will be given:\n1. Original buggy code\n2. The Github Issue name,description of the bug\n3. Trigger test name, code and error message which is causing the issue to be caught\n2. Ground truth patch from benchmark \n3. Generated patch to evaluate\n\n You will systematically analyze patches in a step by step manner using the following structure:\n[Analysis]\n1. Core Functionality and Behavior\n- How does each patch handle the main issue?\n- What are the behavioral differences between patches?\n- How do edge cases differ?\n\n2. Return Value Patterns & Edge Cases\n- Compare return values for key scenarios:\n  * Normal case (valid input)\n  * Missing/null values\n  * Out of bounds values\n  * Invalid inputs\n- Note any differences in behavior\n\n3. Error Handling Patterns\n- Compare exception types and messages\n- Compare validation approaches\n- Identify differences in error handling strategies

In [2]:
import time
import re
import json
import groq
import argparse
from tqdm import tqdm
import os
from src.gen_solution_prompt import sf_construct_prompt
from dotenv import load_dotenv


load_dotenv()

client = groq.Groq()

def make_api_call(messages, max_tokens, is_final_answer=False, custom_client=None):
    global client
    if custom_client is not None:
        client = custom_client
    
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=messages,
        max_tokens=max_tokens,
        temperature=0.2,
    ) 
    return response.choices[0].message.content

print(make_api_call(messages="Hi", max_tokens=100))
                    

BadRequestError: Error code: 400 - {'error': {'message': 'Organization has been restricted. Please reach out to support if you believe this was in error.', 'type': 'invalid_request_error', 'code': 'organization_restricted'}}

In [1]:
print("public TimeSeries createCopy(int start, int end)\n        throws CloneNotSupportedException {\n    if (start < 0) {\n        throw new IllegalArgumentException(\"Requires start >= 0.\");\n    }\n    if (end < start) {\n        throw new IllegalArgumentException(\"Requires start <= end.\");\n    }\n    TimeSeries copy = (TimeSeries) super.clone();\n    copy.minY = Double.NaN;\n    copy.maxY = Double.NaN;\n    copy.data = new java.util.ArrayList();\n    if (this.data.size() > 0) {\n        for (int index = start; index <= end; index++) {\n            TimeSeriesDataItem item\n                    = (TimeSeriesDataItem) this.data.get(index);\n            TimeSeriesDataItem clone = (TimeSeriesDataItem) item.clone();\n            try {\n                copy.add(clone);\n            }\n            catch (SeriesException e) {\n                e.printStackTrace();\n            }\n        }\n    }\n    return copy;\n}")

public TimeSeries createCopy(int start, int end)
        throws CloneNotSupportedException {
    if (start < 0) {
        throw new IllegalArgumentException("Requires start >= 0.");
    }
    if (end < start) {
        throw new IllegalArgumentException("Requires start <= end.");
    }
    TimeSeries copy = (TimeSeries) super.clone();
    copy.minY = Double.NaN;
    copy.maxY = Double.NaN;
    copy.data = new java.util.ArrayList();
    if (this.data.size() > 0) {
        for (int index = start; index <= end; index++) {
            TimeSeriesDataItem item
                    = (TimeSeriesDataItem) this.data.get(index);
            TimeSeriesDataItem clone = (TimeSeriesDataItem) item.clone();
            try {
                copy.add(clone);
            }
            catch (SeriesException e) {
                e.printStackTrace();
            }
        }
    }
    return copy;
}


In [5]:
import os
import json
import glob
from datetime import datetime

def find_missing_evaluations(results_dir="results", output_file="missing_evals.json"):
    """
    Scan all result reports to find bugs without successful evaluations.
    
    Args:
        results_dir (str): Directory containing all run results
        output_file (str): File to write the list of bugs that need re-evaluation
    
    Returns:
        list: List of bug names that need re-evaluation
    """
    print(f"Scanning {results_dir} for result reports...")
    
    # Find all run directories
    run_dirs = glob.glob(os.path.join(results_dir, "run_*"))
    if not run_dirs:
        print(f"No run directories found in {results_dir}")
        return []
    
    print(f"Found {len(run_dirs)} run directories")
    
    # Collect all bugs that need re-evaluation
    bugs_needing_eval = []
    
    # Track which bugs were found in which runs
    bug_sources = {}
    
    for run_dir in run_dirs:
        run_id = os.path.basename(run_dir)
        final_results_file = os.path.join(run_dir, "final_results.json")
        
        if not os.path.exists(final_results_file):
            print(f"Warning: No final_results.json found in {run_dir}")
            continue
        
        try:
            with open(final_results_file, 'r', encoding='utf-8') as f:
                results = json.load(f)
            
            # Check each bug in this run
            for bug_entry in results.get("bugs", []):
                bug_name = bug_entry.get("bug_name")
                
                if not bug_name:
                    continue
                
                # Check if evaluation is missing or failed
                eval_status = bug_entry.get("evaluation", {}).get("status")
                eval_results = bug_entry.get("evaluation", {}).get("results")
                
                # Add to the list if:
                # 1. Evaluation status is not "Success", or
                # 2. Evaluation results are missing/empty
                if eval_status != "Success" or not eval_results:
                    if bug_name not in bugs_needing_eval:
                        bugs_needing_eval.append(bug_name)
                        bug_sources[bug_name] = run_id
                        
                        # Debug information
                        reason = "unknown reason"
                        if eval_status != "Success":
                            reason = f"evaluation status: {eval_status}"
                        elif not eval_results:
                            reason = "missing evaluation results"
                        
                        print(f"Bug {bug_name} needs re-evaluation ({reason}) - from run {run_id}")
            
        except Exception as e:
            print(f"Error processing {final_results_file}: {e}")
    
    # Also check "outputs/val" directory for missing evaluations
    outputs_val_dir = "outputs/val"
    if os.path.exists(outputs_val_dir):
        all_bugs_file = "datasets/defects4j-sf.json"
        if os.path.exists(all_bugs_file):
            try:
                with open(all_bugs_file, 'r', encoding='utf-8') as f:
                    all_bugs = list(json.load(f).keys())
                
                # Get list of bugs that have validation files
                validated_bugs = []
                for val_file in glob.glob(os.path.join(outputs_val_dir, "*_patch_val.json")):
                    bug_name = os.path.basename(val_file).replace("_patch_val.json", "")
                    validated_bugs.append(bug_name)
                
                # Find bugs that don't have validation files
                for bug in all_bugs:
                    if bug not in validated_bugs and bug not in bugs_needing_eval:
                        bugs_needing_eval.append(bug)
                        bug_sources[bug] = "missing_val_file"
                        print(f"Bug {bug} needs re-evaluation (no validation file found)")
            except Exception as e:
                print(f"Error checking validation files: {e}")
    
    print(f"\nFound {len(bugs_needing_eval)} bugs needing re-evaluation")
    
    # Write to output file
    output_data = {
        "timestamp": datetime.now().isoformat(),
        "total_bugs_needing_eval": len(bugs_needing_eval),
        "bugs": bugs_needing_eval,
        "bug_sources": bug_sources
    }
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, indent=2)
    
    print(f"List of bugs needing re-evaluation saved to {output_file}")
    
    return bugs_needing_eval

find_missing_evaluations()

Scanning results for result reports...
Found 23 run directories
Bug Cli-25 needs re-evaluation (evaluation status: None) - from run run_20250228_234822
Bug Cli-26 needs re-evaluation (evaluation status: None) - from run run_20250228_234822
Bug Cli-4 needs re-evaluation (evaluation status: None) - from run run_20250301_001211
Bug Cli-40 needs re-evaluation (evaluation status: None) - from run run_20250301_001211
Bug Closure-101 needs re-evaluation (evaluation status: None) - from run run_20250301_105911
Bug Closure-109 needs re-evaluation (evaluation status: None) - from run run_20250301_105911
Bug Closure-11 needs re-evaluation (evaluation status: None) - from run run_20250301_105911
Bug Closure-123 needs re-evaluation (missing evaluation results) - from run run_20250301_105911
Bug Closure-130 needs re-evaluation (evaluation status: None) - from run run_20250301_105911
Bug Closure-131 needs re-evaluation (evaluation status: None) - from run run_20250301_105911
Bug Closure-145 needs re-

['Cli-25',
 'Cli-26',
 'Cli-4',
 'Cli-40',
 'Closure-101',
 'Closure-109',
 'Closure-11',
 'Closure-123',
 'Closure-130',
 'Closure-131',
 'Closure-145',
 'Closure-150',
 'Closure-152',
 'Closure-17',
 'Closure-21',
 'Closure-25',
 'Closure-35',
 'Closure-57',
 'Closure-58',
 'Closure-65',
 'Closure-62',
 'Closure-7',
 'Closure-96',
 'JacksonCore-23',
 'JacksonCore-25',
 'JacksonCore-26',
 'JacksonCore-8',
 'JacksonCore-7',
 'JacksonDatabind-1',
 'JacksonDatabind-101',
 'JacksonDatabind-17',
 'JacksonDatabind-16',
 'JacksonDatabind-34',
 'JacksonDatabind-33',
 'JacksonDatabind-6',
 'JacksonDatabind-76',
 'JacksonDatabind-88',
 'JacksonDatabind-93',
 'JacksonDatabind-98',
 'JacksonXml-1',
 'Jsoup-64',
 'Jsoup-76',
 'Lang-55',
 'Mockito-28',
 'Time-20',
 'Jsoup-33']

In [1]:
!pip install beautifulsoup4

  Using cached beautifulsoup4-4.13.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.13.3-py3-none-any.whl (186 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
